In [ ]:
import cv2

img = cv2.imread("../input/womens-faces-sorted-by-age-from-18-to-70-years-old/train/22/1001.jpg")

img.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img)
plt.axis("off")

In [ ]:
img_dir = "../input/womens-faces-sorted-by-age-from-18-to-70-years-old/train"

## **Processing the dataset**

In [ ]:
import tensorflow as tf

train_data = tf.keras.preprocessing.image_dataset_from_directory(img_dir,
                                                                                                                                  
                                                                 label_mode = "categorical",
                                                                 image_size = (224,224),
                                                                 batch_size= 32,
                                                                 seed = 42,
                                                                 validation_split = 0.25,
                                                                 subset = "training"
                                                                 )
test_data = tf.keras.preprocessing.image_dataset_from_directory(img_dir,
                                                                label_mode = "categorical",
                                                                image_size = (224,224),
                                                                batch_size = 32,
                                                                seed = 42,
                                                                validation_split = 0.25,
                                                                subset = "validation"
                                                                )

In [ ]:
for image,label in train_data.take(1):
  print(image.shape)
  print(label.shape)

In [ ]:
class_labels = train_data.class_names

len(class_labels)

In [ ]:
# visualizing the train dataset
import tensorflow as tf
import matplotlib.pyplot as plt

plt.figure(figsize=(16,16))
for image,label in train_data.take(1):
  for i in range(18):
    plt.subplot(6,3,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(f"Age : {class_labels[tf.argmax(label[i])]}")
    plt.axis("off")


In [ ]:
# building a autoencoder model
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
                             

                             layers.Conv2D(filters = 20, kernel_size = 2,input_shape= (224,224,3),padding="same",activation= "relu"),
                             layers.BatchNormalization(),
                             layers.MaxPooling2D(pool_size= 2),
                             layers.Conv2D(filters = 20, kernel_size = 2, padding= "same",activation= "relu"),
                             layers.BatchNormalization(),
                             layers.MaxPooling2D(pool_size= 2),
                             layers.Conv2D(filters = 20, kernel_size= 2, padding = "same",activation= "relu"),
                             layers.BatchNormalization(),
                             layers.MaxPooling2D(pool_size = 2),
                             layers.Conv2D(filters = 20, kernel_size= 2, padding = "same",activation= "relu"),



                           layers.Flatten(),
                           
                           layers.Dense(1024, activation = "relu"),                        
                           layers.Dropout(0.3),
                           layers.Dense(1024, activation = "relu"),                          
                           layers.Dropout(0.3),
                           layers.Dense(1024, activation = "relu"),
                                                      
                           layers.Dense(53, activation= "softmax")


                             
])



In [ ]:
model.summary()

In [ ]:
# compiling the model

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [ ]:
# fiting data to the model

model.fit(train_data, 
          steps_per_epoch = len(train_data),      
          epochs = 10,
          validation_data = test_data,
          validation_steps = len(test_data)       

          )

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,16))

for image, label in test_data.take(1):
  
  for i in range(18):
    
    model_pred = model.predict(image)

    plt.subplot(6,3,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(f"Predicted Age : {class_labels[tf.argmax(model_pred[i])]}\nOrignal Age :{class_labels[tf.argmax(label[i])]}")
    plt.subplots_adjust(top = 1.25)
    plt.axis("off")
